## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = range(1,1000+1)
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, numSlices = 10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [11]:
# take
rdd.take(25)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

In [12]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [13]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [14]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([9.42396202e-01, 2.38269333e-01, 2.06363074e+00, 2.19812500e+00,
       1.13971911e+00, 5.92763095e+00, 1.80900068e+00, 3.51302590e+00,
       5.97939043e+00, 5.58498073e+00, 6.00512912e+00, 5.76899436e+00,
       4.27288122e+00, 5.08170584e+00, 1.91981935e+00, 1.48794676e+01,
       2.28469468e+00, 1.40802103e+01, 9.15619792e+00, 4.03634783e+00,
       1.91000940e+00, 1.60064601e+01, 1.03279006e+00, 5.19864015e+00,
       2.27172626e+01, 1.70642183e+01, 2.29876660e+00, 4.21598268e+00,
       2.85515834e+00, 1.61474965e+01, 6.55047080e+00, 1.57314765e+01,
       1.58363643e+01, 1.61478943e+01, 1.56728023e+01, 1.81555194e+01,
       4.60447378e+00, 1.07597403e+00, 2.77516662e+01, 1.54293491e+01,
       3.45795659e+00, 3.86015009e+01, 9.46166706e+00, 1.15327951e+01,
       2.19615132e+01, 1.49223032e+01, 3.28715963e+01, 4.49011462e+01,
       4.38163190e+01, 1.65680897e+01, 4.54679903e+01, 2.30619341e+01,
       8.66008789e+00, 5.82523092e+00, 2.31083150e+01, 1.68713977e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [15]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.top(10)


[933.9459227665743,
 927.8913779115632,
 927.4506063658421,
 906.7203272617018,
 902.5743199057578,
 902.296629008905,
 885.0621353564479,
 882.5417357475433,
 875.9152387912782,
 869.1650427793165]

In [16]:
price_items.take(4)

[0.9423962015713533, 0.23826933300793418, 2.063630744352496, 2.198124999129343]

In [18]:
price_items.count()

1000

In [19]:
price_items.first()

0.9423962015713533

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [34]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [31]:
# perform action to retrieve rdd values
revenue_minus_tax.count()

1000

In [35]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)  # take: returns the first n items in the RDD

[0.8670045054456451,
 0.21920778636729946,
 1.8985402848042963,
 2.0222749991989954,
 1.048541582450758,
 5.453420472121067,
 1.664280628164463,
 3.231983824796045,
 5.501039195191538,
 5.138182267405424]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [36]:
discounted = revenue_minus_tax.map(lambda x: x * .90)

In [37]:
discounted.take(10)

[0.7803040549010806,
 0.1972870077305695,
 1.7086862563238667,
 1.820047499279096,
 0.9436874242056822,
 4.90807842490896,
 1.4978525653480168,
 2.9087854423164403,
 4.950935275672385,
 4.624364040664881]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [38]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[773.3072240507236,
 768.2940609107743,
 767.9291020709172,
 750.7644309726892,
 747.3315368819675,
 747.1016088193734,
 732.831448075139,
 730.7445571989658,
 725.2578177191783,
 719.6686554212741,
 714.5338315831032,
 708.477484322548,
 699.5258108611347,
 697.4202460449028,
 687.4031742782245]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [39]:
discounted.toDebugString()

b'(10) PythonRDD[26] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[8] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [40]:
mapped = price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.9423962015713533, 0.7803040549010806), (0.23826933300793418, 0.1972870077305695), (2.063630744352496, 1.7086862563238667), (2.198124999129343, 1.820047499279096), (1.1397191113595195, 0.9436874242056822), (5.927630947957681, 4.90807842490896), (1.8090006827874598, 1.4978525653480168), (3.5130258965174397, 2.9087854423164403), (5.97939042955602, 4.950935275672385), (5.584980725440677, 4.624364040664881)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [41]:
flat_mapped = price_items.flatMap(lambda x : (x, x*0.92*0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.9423962015713533, 0.7803040549010806, 0.23826933300793418, 0.1972870077305695, 2.063630744352496, 1.7086862563238667, 2.198124999129343, 1.820047499279096, 1.1397191113595195, 0.9436874242056822]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [ ]:
# use the filter function
selected_items = None

# calculate total remaining in inventory 


In [42]:
selected_items = discounted.filter(lambda x: x>300)
selected_items.count()

268

In [43]:
selected_items.collect()

[300.5388968005479,
 312.3888617827471,
 344.2347349032771,
 305.0682683740143,
 337.4796617902419,
 332.77080024091003,
 313.29066427172864,
 364.7929087931017,
 346.4768072501723,
 369.49336486440194,
 337.56411443693474,
 309.55738198620526,
 385.44667603713293,
 305.487920202713,
 357.60844902451765,
 341.4678213930008,
 308.9381676374306,
 378.45359980940464,
 348.87611491249623,
 391.8354830835109,
 352.5003725370659,
 328.2118547090702,
 306.0100947404903,
 402.7264668425411,
 369.5658282851589,
 378.55639407811174,
 357.58200870980477,
 417.07451244028795,
 325.30343509975467,
 410.4855901517202,
 391.06021269100654,
 332.29492892281877,
 315.69706358909934,
 399.7786724349207,
 423.48298897210225,
 393.9884110724272,
 408.45853863028054,
 305.67536825753336,
 327.5300755650565,
 343.6829661642813,
 441.59531984242733,
 350.0341484717842,
 312.8282147775178,
 345.437977174382,
 403.8172560810477,
 445.7058774315186,
 409.3282644746881,
 332.0858972821963,
 343.69935541461956,
 

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [44]:
selected_items.reduce(lambda x,y: x+y)

126176.6151393906

In [45]:
sum([300.5388968005479,
 312.3888617827471,
 344.2347349032771,
 305.0682683740143,
 337.4796617902419,
 332.77080024091003,
 313.29066427172864,
 364.7929087931017,
 346.4768072501723,
 369.49336486440194,
 337.56411443693474,
 309.55738198620526,
 385.44667603713293,
 305.487920202713,
 357.60844902451765,
 341.4678213930008,
 308.9381676374306,
 378.45359980940464,
 348.87611491249623,
 391.8354830835109,
 352.5003725370659,
 328.2118547090702,
 306.0100947404903,
 402.7264668425411,
 369.5658282851589,
 378.55639407811174,
 357.58200870980477,
 417.07451244028795,
 325.30343509975467,
 410.4855901517202,
 391.06021269100654,
 332.29492892281877,
 315.69706358909934,
 399.7786724349207,
 423.48298897210225,
 393.9884110724272,
 408.45853863028054,
 305.67536825753336,
 327.5300755650565,
 343.6829661642813,
 441.59531984242733,
 350.0341484717842,
 312.8282147775178,
 345.437977174382,
 403.8172560810477,
 445.7058774315186,
 409.3282644746881,
 332.0858972821963,
 343.69935541461956,
 356.03698839495684,
 336.74915835255854,
 444.4872447480037,
 383.0306665760281,
 432.22838251856706,
 455.4513318524367,
 324.4982470508329,
 377.6726429179994,
 404.07049793512215,
 477.8705258331732,
 493.4019758425256,
 477.54447317007435,
 350.0823368795118,
 495.4048019542136,
 508.4358101505339,
 408.7482443975164,
 473.1181604230252,
 327.0196654152253,
 374.6800783682816,
 383.17575948244206,
 490.8087939514705,
 326.78518045626026,
 465.37141287606266,
 438.5258854894646,
 522.4616665590976,
 461.74277492758506,
 462.7251400289864,
 514.816104555525,
 504.76739473425596,
 444.5851689119262,
 353.06226472748676,
 401.86001209961927,
 445.5269795506618,
 404.5462417612659,
 489.73720507665456,
 498.8283689567929,
 535.35689853779,
 331.7557613056702,
 367.8826318207738,
 368.97009908809554,
 446.2274264578752,
 327.9633426179393,
 420.3643000027205,
 342.26383117668973,
 461.7730246852675,
 533.4577036604876,
 300.79412633240645,
 376.7314445445373,
 558.3203450292015,
 530.590083918518,
 475.0377865881469,
 340.1764533788285,
 351.94870140010323,
 450.2470358819819,
 578.0753710626577,
 572.7860107301788,
 569.8281978212436,
 329.99771659629357,
 539.2963022329171,
 437.26170561031995,
 484.88488152444194,
 392.51127928073134,
 576.3556590036844,
 344.22756936454283,
 500.8726128881078,
 372.1628983327631,
 581.2142537498424,
 390.26219608639235,
 359.726567713965,
 533.6681802450124,
 385.3500445549943,
 554.3838871429066,
 399.00933266314064,
 522.9873151496953,
 473.64405306454466,
 399.6088062626639,
 345.58063638515125,
 473.8098994276757,
 323.6379401360336,
 419.8017682202155,
 553.5298395611022,
 483.03160698844886,
 567.4461628313545,
 629.111030993223,
 486.17548076552083,
 551.156913418884,
 348.65741922413025,
 399.71016025773844,
 601.7106034292453,
 318.70611291106144,
 371.6930624716537,
 450.45770881324853,
 540.1970104977968,
 396.23459729727495,
 336.1303490126624,
 450.34398208733614,
 620.2958922237151,
 564.8024422056359,
 331.0476672658477,
 622.9987112950016,
 655.32006780858,
 609.4508508759112,
 597.9541715347418,
 604.6969357928964,
 629.0373566086421,
 624.481135322294,
 619.5361623279481,
 455.6572732798685,
 443.52646024438826,
 658.6886347367744,
 529.7094955625469,
 314.97174216000724,
 672.3921913210645,
 606.001315681949,
 616.9022311940587,
 443.73888401351013,
 492.6447894188638,
 604.2975509821362,
 357.97623167771167,
 362.1664829232815,
 397.05629649008176,
 650.2880810772284,
 382.4460512193972,
 593.9270423367368,
 464.0167009726536,
 552.7766078961469,
 616.9118844959588,
 653.4964384066516,
 615.2969038190989,
 606.2917960692439,
 697.4202460449028,
 428.91253336192864,
 329.4593286209641,
 317.54046958001925,
 687.4031742782245,
 675.0587083012866,
 478.4793437876747,
 354.4611990402575,
 432.71572466171386,
 566.648708496439,
 719.6686554212741,
 391.88637905103883,
 538.6620666823736,
 405.9791850840507,
 730.7445571989658,
 416.50691681503554,
 478.1046440169053,
 566.3789594923497,
 639.5068170353941,
 483.9670317005963,
 417.05070716182536,
 465.1644781920983,
 623.0262805630618,
 506.72120497957025,
 350.91265035347504,
 571.656094491822,
 560.0117774743047,
 593.5113145874742,
 354.7501673573271,
 511.1990440080961,
 432.6548964602531,
 656.0502134269426,
 545.1214020193958,
 714.5338315831032,
 594.5061450845319,
 675.0385601012138,
 332.17576881262,
 484.876831028948,
 379.71192733844447,
 376.42832682846824,
 583.4202709005597,
 500.6691850535815,
 632.620190170361,
 499.25474042208845,
 747.3315368819675,
 572.7558756548839,
 699.5258108611347,
 773.3072240507236,
 645.8673021697482,
 767.9291020709172,
 412.147900676266,
 725.2578177191783,
 443.9920756020741,
 750.7644309726892,
 529.4085345726862,
 563.7366370856153,
 593.1323242289628,
 671.2382630065948,
 537.1882157145941,
 487.8462251011047,
 301.20246462174146,
 569.2460654325187,
 363.55983561960534,
 620.2610294584729,
 585.8222820674079,
 453.21035723379174,
 440.53708959969833,
 458.4811762680093,
 732.831448075139,
 373.08547868423796,
 305.96418168412936,
 544.9243303408571,
 657.655829271929,
 708.477484322548,
 520.112869115526,
 509.43134359307464,
 422.9232168145285,
 747.1016088193734,
 768.2940609107743,
 354.1795388109808,
 508.11750933262374,
 530.1724242302547,
 584.4211094533961,
 679.3419027864372,
 368.9547959921364,
 542.3834149166711,
 510.11112904850273,
 669.4647593933229,
 312.6194892100416])

126176.61513939057

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [46]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(24, 300.5388968005479),
 (38, 312.3888617827471),
 (28, 344.2347349032771),
 (45, 305.0682683740143),
 (12, 337.4796617902419),
 (16, 332.77080024091003),
 (23, 313.29066427172864)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [47]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(50, 1300.063009656088),
 (10, 2984.0736370859495),
 (30, 3151.488879690557),
 (20, 3810.7587473146245),
 (40, 1299.6208154519852),
 (41, 4704.051433716914),
 (11, 2807.516464598505),
 (1, 3443.841010874714),
 (31, 1184.7300258629953),
 (21, 2369.988301454176)]

In [48]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(41, 4704.051433716914),
 (37, 4605.7123923904965),
 (47, 4482.334472348438),
 (12, 4157.643398770031),
 (8, 4055.051897837876),
 (3, 3872.2589859832774),
 (20, 3810.7587473146245),
 (24, 3501.5943395051554),
 (45, 3456.112676256841),
 (1, 3443.841010874714),
 (38, 3430.3308509123194),
 (30, 3151.488879690557),
 (7, 3132.3629631720314),
 (2, 3080.7907020842076),
 (27, 3048.2509900489167),
 (10, 2984.0736370859495),
 (11, 2807.516464598505),
 (32, 2795.343519838679),
 (42, 2708.9260881068058),
 (17, 2685.9390825907726),
 (33, 2661.9583712051126),
 (25, 2637.295356713995),
 (34, 2625.299260441254),
 (29, 2600.1683172749345),
 (35, 2477.701156696989),
 (21, 2369.988301454176),
 (5, 2359.9640097783426),
 (36, 2338.3154638214214),
 (15, 2293.540616407901),
 (9, 2253.1326687384258),
 (16, 2252.64612267634),
 (6, 2244.896918919041),
 (48, 2194.1383937049523),
 (23, 2108.743915423514),
 (39, 1962.1379174786177),
 (28, 1936.2527615809731),
 (18, 1936.0497583215893),
 (22, 1814.8955071509492),


Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [49]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(16, 10),
 (17, 9),
 (27, 9),
 (6, 8),
 (9, 8),
 (24, 8),
 (12, 7),
 (29, 7),
 (42, 7),
 (45, 7),
 (49, 7),
 (19, 7),
 (37, 7),
 (48, 7),
 (20, 7),
 (4, 6),
 (5, 6),
 (26, 6),
 (3, 6),
 (14, 6),
 (31, 6),
 (11, 6),
 (50, 6),
 (7, 6),
 (34, 6),
 (25, 6),
 (23, 5),
 (43, 5),
 (33, 5),
 (35, 5),
 (39, 5),
 (18, 5),
 (32, 5),
 (15, 5),
 (46, 4),
 (1, 4),
 (13, 4),
 (30, 4),
 (22, 4),
 (40, 4),
 (41, 3),
 (44, 3),
 (38, 3),
 (28, 2),
 (21, 2),
 (36, 2),
 (10, 2),
 (47, 2),
 (8, 2),
 (2, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
